---

<center>

# **Python pour la Science des Données**

### *Traitement des Données*

</center>

---

<center>

## **📖 Introduction**

</center>

---


Le prétraitement des données peut se résumer en 4 opérations essentielles : **filtrage, jointure, tri et regroupement**.

La structure **DataFrame** est devenue la norme pour la manipulation de données car, dans la plupart des cas, il suffit de répéter ou de combiner ces quatre opérations.

Dans cet exercice, vous apprendrez à utiliser ces 4 méthodes de prétraitement des données.

Avant de commencer ce notebook, exécutez la cellule suivante afin de récupérer le travail effectué dans les notebooks précédents.

In [1]:
### Import ###

import pandas as pd

# Importer le jeu de données
transactions = pd.read_csv("transactions.csv", sep=';', index_col="transaction_id")

# Supprimer les doublons
transactions = transactions.drop_duplicates(keep='first')

# Renommer les colonnes
nouveaux_noms = {'Store_type': 'type_magasin',
                 'Qty': 'quantite',
                 'Rate': 'prix',
                 'Tax': 'taxe'}

transactions = transactions.rename(nouveaux_noms, axis=1)

### Gestion des valeurs manquantes (NAs) ###

# Remplacer les NaN dans 'prod_subcat_code' par -1
transactions['prod_subcat_code'] = transactions['prod_subcat_code'].fillna(-1).astype("int")

# Obtenir la valeur la plus fréquente de 'store_type'
mode_type_magasin = transactions['type_magasin'].mode()

# Remplacer les NaN dans 'type_magasin' par sa valeur la plus fréquente
transactions['type_magasin'] = transactions['type_magasin'].fillna(transactions['type_magasin'].mode()[0])

# Supprimer les lignes où 'prix', 'taxe' et 'total_amt' sont tous manquants
transactions = transactions.dropna(axis=0, how='all', subset=['prix', 'taxe', 'total_amt'])

---

<center>

## **📖 Filtrer un DataFrame avec des opérateurs binaires**

</center>

---


Le filtrage consiste à sélectionner un sous-ensemble de lignes d'un DataFrame qui satisfont une condition.  
C'est ce que nous appelions précédemment *indexation conditionnelle*, mais le terme *filtrage* est le plus couramment utilisé en gestion de bases de données.  

Nous ne pouvons pas utiliser les opérateurs logiques `and` et `or` lors du filtrage avec plusieurs conditions.  
Ces opérateurs créent une ambiguïté que **pandas** ne peut pas gérer lors du filtrage des lignes.  

Les opérateurs adaptés au filtrage avec plusieurs conditions sont les **opérateurs binaires** :

- L'opérateur 'and' : `&`  
- L'opérateur 'or' : `|`  
- L'opérateur 'not' : `~`  

Ces opérateurs sont similaires aux opérateurs logiques, mais leurs méthodes d’évaluation ne sont pas les mêmes.  

---

### L'opérateur 'and' : `&`

L'opérateur `&` est utilisé pour filtrer un DataFrame avec plusieurs conditions qui doivent toutes être satisfaites simultanément.  

**Exemple :**  

Considérons le DataFrame `df` suivant contenant des informations sur des appartements à Paris :

| neighborhood     | year  | surface |
|------------------|-------|---------|
| 'Champs-Elysées' | 1979  | 70      |
| 'Europe'         | 1850  | 110     |
| 'Père-Lachaise'  | 1935  | 55      |
| 'Bercy'          | 1991  | 30      |

Si nous voulons trouver un appartement construit en **1979** et ayant une surface supérieure à **60 m²**, nous pouvons filtrer les lignes de `df` avec le code suivant :


```python
# Filtrer le DataFrame avec les 2 conditions précédentes
print(df[(df['year'] == 1979) & (df['surface'] > 60)])

>>>       neighborhood   year  surface
>>> 0   Champs-Elysées  1979       70
```
Les conditions doivent être entourées de parenthèses pour éviter toute ambiguïté dans l’ordre d’évaluation.  
En effet, si les conditions ne sont pas correctement séparées, nous obtiendrons l’erreur suivante :

```python
print(df[df['year'] == 1979 & df['surface'] > 60])

>>> ValueError: The truth value of a Series is ambiguous.
>>> Use a.empty, a.bool(), a.item(), a.any() or a.all().
```

### L'opérateur 'or' : `|`

L'opérateur `|` est utilisé pour filtrer un DataFrame avec plusieurs conditions où au moins l'une d'elles doit être satisfaite.  

**Exemple :**  

Prenons le même DataFrame `df` :  

| neighborhood       | year | surface (m²) |
|--------------------|------|--------------|
| 'Champs-Elysées'   | 1979 | 70           |
| 'Europe'           | 1850 | 110          |
| 'Père-Lachaise'    | 1935 | 55           |
| 'Bercy'            | 1991 | 30           |

Si nous voulons trouver un appartement construit **après 1900** ou situé dans le quartier **Père-Lachaise**, nous pouvons filtrer les lignes de `df` avec le code suivant :

```python
# Filtrer le DataFrame avec les 2 conditions précédentes
print(df[(df['year'] > 1900) | (df['neighborhood'] == 'Père-Lachaise')])

>>>     neighborhood    year  surface
>>> 0  Champs-Elysées   1979       70
>>> 2  Père-Lachaise    1935       55
>>> 3           Bercy   1991       30
````

### L'opérateur 'not' : `~`

L'opérateur `~` est utilisé pour filtrer un DataFrame selon une condition dont la **négation** doit être satisfaite.  

**Exemple :**  

Prenons le même DataFrame `df` :   

| neighborhood       | year | surface (m²) |
|--------------------|------|--------------|
| 'Champs-Elysées'   | 1979 | 70           |
| 'Europe'           | 1850 | 110          |
| 'Père-Lachaise'    | 1935 | 55           |
| 'Bercy'            | 1991 | 30           |

Si nous voulons un appartement **qui n’est pas situé dans le quartier Bercy**, nous pouvons filtrer `df` comme suit :

```python
# Filtrer le DataFrame pour exclure le quartier Bercy
print(df[~(df['neighborhood'] == 'Bercy')])

>>>     neighborhood    year  surface
>>> 0  Champs-Elysées   1979       70
>>> 1          Europe   1850      110
>>> 2  Père-Lachaise    1935       55
```

<center>

### **🔍 Exemple : Filtrage avec conditions**

</center>

---

- (a) Afficher les 5 premières lignes du DataFrame `transactions`.  
- (b) À partir de `transactions`, créer un DataFrame nommé `e_shop` ne contenant que les transactions réalisées dans les magasins de type `'e-Shop'` avec un montant total supérieur à 5000 (colonnes : `store_type` et `total_amt`).  
- (c) De la même manière, créer un DataFrame nommé `teleshop` ne contenant que les transactions réalisées dans les magasins de type `'TeleShop'` avec un montant total supérieur à 5000.  
- (d) Parmi ces deux types de magasins, lequel a le plus grand nombre de transactions supérieures à 5000 € ?

In [ ]:
# TODO

<center>

### **🔍 Exemple : Gestion des valeurs manquantes**

</center>

---

- (a) Importer les données des fichiers `'customer.csv'` et `'prod_cat_info.csv'` dans deux DataFrames nommés respectivement `customer` et `prod_cat_info`.  

- (b) Les colonnes `Gender` et `city_code` dans `customer` contiennent chacune deux valeurs manquantes. Remplacez-les par leur mode en utilisant les méthodes `fillna` et `mode`.



In [ ]:
# TODO

## Combiner des DataFrames avec `concat`

La fonction `concat` du module **pandas** permet de concaténer plusieurs DataFrames, c’est-à-dire de les empiler **verticalement** ou **horizontalement**.  

La signature de la fonction est la suivante : `pandas.concat(objs, axis=...)`  

- Le paramètre `objs` contient la liste des DataFrames à concaténer.  
- Le paramètre `axis` indique s’il faut concaténer **verticalement** (`axis=0`) ou **horizontalement** (`axis=1`).  

Lorsque le nombre de lignes ou de colonnes des DataFrames ne correspond pas, la fonction `concat` remplit les cellules manquantes avec `NaN`, comme illustré ci-dessous.

<center>

### **🔍 Exemple : Concaténer des DataFrames**

</center>

---

- (a) Séparer les variables (colonnes) du DataFrame `transactions` en deux, avec la moitié des colonnes dans un DataFrame nommé `part_1` et l’autre moitié dans un DataFrame nommé `part_2`.  
- (b) Reconstruire `transactions` dans un DataFrame nommé `union` en concaténant `part_1` et `part_2`.  
- (c) Que se passe-t-il si nous concaténons `part_1` et `part_2` en utilisant l’argument `axis=0` ?

In [ ]:
# TODO

## Fusionner des DataFrames avec la méthode `merge`

Deux DataFrames peuvent être fusionnés s'ils ont une colonne en commun.  
Cela se fait à l'aide de la méthode `merge` d'un DataFrame, qui a la signature suivante :

`merge(right, on, how, ...)`

- Le paramètre `right` est le DataFrame avec lequel fusionner le DataFrame appelant.  
- Le paramètre `on` est le nom des colonnes des DataFrames qui serviront de référence pour la fusion. Ces colonnes doivent exister dans les deux DataFrames.  
- Le paramètre `how` spécifie le type de jointure à effectuer pour fusionner les DataFrames. Ses valeurs sont basées sur la syntaxe SQL des jointures.  

Le paramètre `how` peut prendre 4 valeurs (`'inner'`, `'outer'`, `'left'`, `'right'`), illustrées avec les deux DataFrames suivants `Persons` et `Vehicle` :

**Persons**

| Name     | Car        |
|----------|------------|
| Lila     | Twingo     |
| Tiago    | Clio       |
| Berenice | C4 Cactus  |
| Joseph   | Twingo     |
| Kader    | Swift      |
| Romy     | Scenic     |

**Vehicle**

| Car       | Price  |
|-----------|--------|
| Twingo    | 11000  |
| Swift     | 14500  |
| C4 Cactus | 23000  |
| Clio      | 16000  |
| Prius     | 30000  |

- `'inner'` : C'est la valeur par défaut de `how`. Une jointure interne retourne uniquement les lignes où les valeurs des colonnes communes existent dans les deux DataFrames. Ce type de jointure est souvent déconseillé car il peut générer beaucoup de valeurs manquantes, mais il produit **aucun NaN**.  

    Exemple : `Persons.merge(right=Vehicle, on='Car', how='inner')`  

- `'outer'` : Une jointure externe fusionne toutes les lignes des deux DataFrames. Aucune ligne n’est supprimée. Cette méthode peut générer beaucoup de NaN.  

    Exemple : `Persons.merge(right=Vehicle, on='Car', how='outer')`  

- `'left'` : Une jointure gauche retourne toutes les lignes du DataFrame de gauche et les complète avec les lignes correspondantes du DataFrame de droite selon la colonne commune.  

    Exemple : `Persons.merge(right=Vehicle, on='Car', how='left')`  

- `'right'` : Une jointure droite retourne toutes les lignes du DataFrame de droite et les complète avec les lignes correspondantes du DataFrame de gauche selon la colonne commune.  

    Exemple : `Persons.merge(right=Vehicle, on='Car', how='right')`  

Effectuer une jointure gauche, droite ou externe suivie de `dropna(how='any')` est équivalent à une jointure interne.


<center>

### **🔍 Example: Merging transactions with customer data**

</center>

---

The `customer` DataFrame contains information about clients corresponding to the `'cust_id'` column in `transactions`.  

The `'customer_Id'` column in the `customer` DataFrame will allow us to join `transactions` and `customer`.  
This will enrich the `transactions` dataset with additional information.  

- (a) Using the `rename` method and a dictionary, rename the `'customer_Id'` column in the `customer` DataFrame to `'cust_id'`.  
- (b) Using the `merge` method, perform a **left join** between `transactions` and `customer` on the `'cust_id'` column. Name the resulting DataFrame `fusion`.  
- (c) Did the merge produce any `NaN` values?  
- (d) Display the first rows of `fusion`. What are the new columns?

In [ ]:
# TODO

## Réinitialiser et définir l’index d’un DataFrame

La fusion a réussi et n’a produit aucun NaN. Cependant, l’index du DataFrame obtenu n’est plus la colonne `'transaction_id'` et a été réinitialisé à l’index par défaut (0, 1, 2, ...).  

Il est possible de **redéfinir l’index** d’un DataFrame en utilisant la méthode `set_index`.  

Cette méthode peut prendre en argument :

- Le nom d’une colonne à utiliser comme index.  
- Un tableau Numpy ou une Series pandas ayant le même nombre de lignes que le DataFrame appelant.  

**Exemple :**  

Soit le DataFrame `df` suivant :

| Name     | Car        |
|----------|------------|
| Lila     | Twingo     |
| Tiago    | Clio       |
| Berenice | C4 Cactus  |
| Joseph   | Twingo     |
| Kader    | Swift      |
| Romy     | Scenic     |

Nous pouvons définir la colonne `'Name'` comme nouvel index :

```python
df = df.set_index('Name')
```

Cela produira le DataFrame suivant :

| Name     | Car        |
|----------|------------|
| Lila     | Twingo     |
| Tiago    | Clio       |
| Berenice | C4 Cactus  |
| Joseph   | Twingo     |
| Kader    | Swift      |
| Romy     | Scenic     |

Nous pouvons également définir l’index à l’aide d’un tableau Numpy, d’une Series, etc.

```python
# NOuveau index à utiliser
new_index = ['10000' + str(i) for i in range(6)]
print(new_index)
>>> ['100000', '100001', '100002', '100003', '100004', '100005']

index_series = pd.Series(new_index)
df = df.set_index(index_series)
```

Cela produira le DataFrame suivant :

|       | Name     | Car        |
|-------|----------|------------|
| 100000| Lila     | Twingo     |
| 100001| Tiago    | Clio       |
| 100002| Berenice | C4 Cactus  |
| 100003| Joseph   | Twingo     |
| 100004| Kader    | Swift      |
| 100005| Romy     | Scenic     |

Pour revenir à l’indexation numérique par défaut, utilisez la méthode `reset_index` du DataFrame :

```python
df = df.reset_index()
```

L’ancien index n’est pas supprimé. Une nouvelle colonne est créée contenant l’ancien index :

|       | index    |   Name     |   Car     |
|-------|----------|------------|-----------|
| 0     | 100000   | Lila       | Twingo    |
| 1     | 100001   | Tiago      | Clio      |
| 2     | 100002   | Berenice   | C4 Cactus |
| 3     | 100003   | Joseph     | Twingo    |
| 4     | 100004   | Kader      | Swift     |
| 5     | 100005   | Romy       | Scenic    |


<center>

### **🔍 Exemple : Rétablir l’index après une fusion**

</center>

---

La fusion entre `transactions` et `customer` a supprimé l’index de `transactions`.  

L’index d’un DataFrame peut être récupéré grâce à son attribut `.index`.  

- (a) Récupérer l’index de `transactions` et l’utiliser pour définir l’index de `fusion`.

In [ ]:
# TODO

## Trier un DataFrame : méthodes `sort_values` et `sort_index`

La méthode `sort_values` permet de trier les lignes d’un DataFrame en fonction des valeurs d’une ou plusieurs colonnes.  

La signature de la méthode est : `sort_values(by, ascending, ...)`

- Le paramètre `by` spécifie la ou les colonnes sur lesquelles effectuer le tri.  
- Le paramètre `ascending` est un booléen (`True` ou `False`) qui détermine si le tri est croissant ou décroissant. Par défaut, il est à `True`.  

**Exemple :**  

Considérons le DataFrame `df` suivant décrivant des étudiants :

| FirstName | Grade | BonusPoints |
|-----------|-------|-------------|
| 'Amelie'  | A     | 1           |
| 'Marin'   | F     | 1           |
| 'Pierre'  | A     | 2           |
| 'Zoe'     | C     | 1           |

Tout d’abord, nous allons trier selon une seule colonne, par exemple la colonne `'BonusPoints'` :

```python
# Sort the DataFrame df by the 'BonusPoints' column
df_sorted = df.sort_values(by='BonusPoints', ascending=True)
```
Le résultat sera le suivant :

| FirstName | Grade | BonusPoints |
|-----------|-------|-------------|
| 'Amelie'  | A     | 1           |
| 'Marin'   | F     | 1           |
| 'Zoe'     | C     | 1           |
| 'Pierre'  | A     | 2           |

Les lignes du DataFrame `df_sorted` sont donc triées dans l’ordre croissant de la colonne `'BonusPoints'`.  
Cependant, si l’on observe la colonne `'Grade'`, on remarque qu’elle n’est pas triée alphabétiquement pour les lignes ayant la même valeur de `'BonusPoints'`.  

Nous pouvons corriger cela en triant également par la colonne `'Grade'` :


```python
# Trier le DataFrame df par 'BonusPoints' et, en cas d'égalité, par 'Grade'
df_sorted = df.sort_values(by=['BonusPoints', 'Grade'], ascending=True)
```
Le résultat sera le suivant :

La méthode `sort_index` permet de trier un DataFrame en fonction de son index.  
Lorsque l’index est l’index numérique par défaut, cette méthode n’est pas très utile.  
Elle est donc souvent combinée avec la méthode `set_index` de pandas, comme nous l’avons vu précédemment.  

**Exemple :**

```python
# Définir la colonne 'Grade' comme index de df
df = df.set_index('Grade')

# Trier le DataFrame df selon son index
df = df.sort_index()
```

Cela produit le DataFrame suivant :

| Grade | FirstName | BonusPoints |
|-------|-----------|-------------|
| A     | 'Amelie'  | 1           |
| A     | 'Pierre'  | 2           |
| C     | 'Zoe'     | 1           |
| F     | 'Marin'   | 1           |

Considérons les deux DataFrames suivants contenant des données de location de bateaux.  

**DataFrame des Bateaux (`boats`) :**

| BoatName   | Color  | ReservationNumber | NumberOfReservations |
|------------|--------|-----------------|--------------------|
| Julia      | blue   | 2               | 34                 |
| Siren      | green  | 3               | 10                 |
| Sea Sons   | red    | 6               | 20                 |
| Hercules   | blue   | 1               | 41                 |
| Cesar      | yellow | 4               | 12                 |
| Minerva    | green  | 5               | 16                 |

**DataFrame des Clients (`clients`) :**

| ClientID | ClientName | ReservationID |
|----------|------------|---------------|
| 91       | Marie      | 1             |
| 154      | Anna       | 2             |
| 124      | Yann       | 3             |
| 320      | Lea        | 7             |
| 87       | Marc       | 9             |
| 22       | Yassine    | 10            |



In [ ]:
# Exécuter la cellule suivante pour créer ces DataFrames.
# Définir les dictionnaires
data_boats = {
    'BoatName': ['Julia', 'Siren', 'Sea Sons', 'Hercules', 'Cesar', 'Minerva'],
    'Color': ['blue', 'green', 'red', 'blue', 'yellow', 'green'],
    'ReservationNumber': [2, 3, 6, 1, 4, 5],
    'NumberOfReservations': [34, 10, 20, 41, 12, 16]
}

data_clients = {
    'ClientID': [91, 154, 124, 320, 87, 22],
    'ClientName': ['Marie', 'Anna', 'Yann', 'Lea', 'Marc', 'Yassine'],
    'ReservationID': [1, 2, 3, 7, 9, 10]
}

# Créer les DataFrames
boats = pd.DataFrame(data_boats)
clients = pd.DataFrame(data_clients)

<center>

### **🔍 Exemple : Joindre les données des bateaux et des clients**

</center>

---

Nous voulons déterminer facilement quel client a réservé les bateaux du DataFrame `bateaux`.  
Pour cela, il suffit de fusionner les DataFrames.  

- (a) Renommer la colonne `'ReservationNumber'` dans `bateaux` en `'ReservationID'` à l’aide de la méthode `rename`.  
- (b) Dans un DataFrame nommé `bateaux_clients`, effectuer une **jointure gauche** entre `bateaux` et `clients`.  
- (c) Définir la colonne `'BoatName'` comme index du DataFrame `bateaux_clients`.  
- (d) À l’aide de la méthode `loc`, qui permet d’indexer un DataFrame, déterminer qui a réservé les bateaux `'Julia'` et `'Siren'`.  
- (e) En utilisant la méthode `isna` appliquée à la colonne `'ClientName'`, déterminer quels bateaux n’ont pas été réservés.  
- (f) Le nombre de fois qu’un bateau a été réservé jusqu’à présent est donné dans la colonne `'NumberOfReservations'`.  
En utilisant la méthode `sort_values`, déterminer le nom du client ayant réservé le **bateau bleu** avec le plus grand nombre de réservations.

In [ ]:
# TODO

## Grouper les éléments d’un DataFrame : méthodes `groupby`, `agg` et `crosstab`

La méthode `groupby` permet de regrouper les lignes d’un DataFrame qui partagent une même valeur dans une colonne.  

Cette méthode **ne** renvoie **pas** un DataFrame.  
L’objet renvoyé par `groupby` est de la classe `DataFrameGroupBy`.  

Cette classe permet d’effectuer des opérations telles que le calcul de statistiques (somme, moyenne, maximum, etc.) pour chaque catégorie de la colonne utilisée pour le regroupement.  

La structure générale d’une opération `groupby` est la suivante :

1. Séparer les données (Split).  
2. Appliquer une fonction (Apply).  
3. Combiner les résultats (Combine).  

**Exemple :**  

Supposons que les bateaux du DataFrame `boats` soient tous identiques et aient le même âge.  
Nous voulons déterminer si la couleur d’un bateau influence son nombre de réservations.  
Pour cela, nous allons calculer le nombre moyen de réservations par bateau pour chaque couleur :

- Séparer les bateaux par couleur.  
- Calculer le nombre moyen de réservations (`mean`).  
- Combiner les résultats dans un DataFrame pour une comparaison facile.  

Nous pouvons utiliser `groupby` suivi de `mean` pour obtenir le résultat.  

Toutes les méthodes statistiques courantes (`count`, `mean`, `max`, etc.) peuvent être utilisées après `groupby`.  
Elles ne s’appliqueront qu’aux colonnes de types compatibles.  

Il est également possible de spécifier pour chaque colonne quelle fonction doit être appliquée à l’étape "Apply" d’une opération `groupby`.  
Pour cela, utilisez la méthode `agg` de l’objet `DataFrameGroupBy`, en fournissant un dictionnaire où chaque clé est un nom de colonne et la valeur est la fonction à appliquer.  

**Exemple :**  

Considérons le DataFrame `transactions` :


| transaction_id | cust_id | tran_date  | prod_subcat_code | prod_cat_code | qty  | rate  | tax    | total_amt | store_type |
|----------------|---------|-----------|-----------------|---------------|------|-------|--------|-----------|------------|
| 80712190438    | 270351  | 28-02-14  | 1               | 1             | -5   | -772  | 405.3  | -4265.3   | e-Shop     |
| 29258453508    | 270384  | 27-02-14  | 5               | 3             | -5   | -1497 | 785.925| -8270.92  | e-Shop     |
| 51750724947    | 273420  | 24-02-14  | 6               | 5             | -2   | -791  | 166.11 | -1748.11  | TeleShop   |
| 93274880719    | 271509  | 24-02-14  | 11              | 6             | -3   | -1363 | 429.345| -4518.35  | e-Shop     |
| 51750724947    | 273420  | 23-02-14  | 6               | 5             | -2   | -791  | 166.11 | -1748.11  | TeleShop   |

Nous voulons, pour chaque client (`cust_id`) :

- Pour la colonne `total_amt` : le montant minimum, maximum et total dépensé.  
- Pour la colonne `store_type` : le nombre de types de magasins différents dans lesquels le client a effectué une transaction.  

Nous pouvons effectuer ces calculs en utilisant une opération `groupby` :

1. Séparer les transactions par ID de client.  
2. Pour `total_amt`, calculer `min`, `max` et `sum`. Pour `store_type`, compter le nombre de catégories uniques.  
3. Combiner les résultats dans un DataFrame.

Pour trouver le nombre de catégories uniques pour `store_type`, nous pouvons utiliser la fonction lambda suivante :

```python
import numpy as np

n_modalities = lambda store_type: len(np.unique(store_type))
```

- La fonction lambda doit prendre une colonne en argument et retourner un nombre.  
- La fonction `np.unique` détermine les valeurs uniques présentes dans une séquence.  
- La fonction `len` compte le nombre d’éléments dans une séquence.  

Ainsi, cette fonction nous permet de déterminer le nombre de catégories uniques pour la colonne `store_type`.  

Pour appliquer ces fonctions dans une opération `groupby`, nous utilisons un dictionnaire où les clés sont les colonnes à traiter et les valeurs sont les fonctions à appliquer.

```python
functions_to_apply = {
   # Les méthodes statistiques standard peuvent être spécifiées sous forme de chaînes de caractères
    'total_amt': ['min', 'max', 'sum'],
    'store_type': n_modalities
}
```

Ce dictionnaire peut maintenant être utilisé avec la méthode `agg` :


```python
transactions.groupby('cust_id').agg(functions_to_apply)
```

Cela produit le `DataFrameGroupBy` suivant :

            total_amount          store_type
| cust_id | min      | max     | sum     | lambda  |
|---------|----------|---------|---------|---------|
| 266783  | -5838.82 | 5838.82 | 3113.89 |   2     |
| 266784  | 442      | 4279.66 | 5694.07 |   3     |
| 266785  | -6828.9  | 6911.77 | 21613.8 |   3     |
| 266788  | 1312.74  | 1927.12 | 6092.97 |   3     |
| 266794  | -135.915 | 4610.06 | 27981.9 |   4     |
           


<center>

### **🔍 Exemple : Grouper par client pour analyser les quantités**

</center>

---

- (a) À l’aide d’une opération `groupby`, déterminer pour chaque client, en se basant sur la quantité d’articles achetés dans une transaction (colonne `qty`) :

  - La quantité maximale.  
  - La quantité minimale.  
  - La quantité médiane.  

  Vous devez filtrer les transactions pour ne conserver que celles avec des quantités positives.  
  Pour cela, vous pouvez utiliser l’indexation conditionnelle (`qty[qty > 0]`) dans une fonction lambda.

In [ ]:
# TODO

Une autre manière de regrouper et de résumer les données est d’utiliser la fonction `crosstab` de pandas, qui, comme son nom l’indique, sert à réaliser des tableaux croisés de colonnes d’un DataFrame.  

Elle permet de visualiser la fréquence d’occurrence des paires de catégories dans un DataFrame.  

**Exemple :**  

Dans le DataFrame `transactions`, nous voulons savoir quelles paires catégorie / sous-catégorie sont les plus fréquentes (colonnes `prod_cat_code` et `prod_subcat_code`).  

La fonction `crosstab` de pandas peut être utilisée comme suit :

```python
colonne1 = transactions['prod_cat_code']
colonne2 = transactions['prod_subcat_code']
pd.crosstab(colonne1, colonne2)
```

Cette commande produit le DataFrame suivant :

prod_subcat_code

| prod_cat_code | -1 | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 9   | 10  | 11  | 12  |
|---------------|----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
| 1             | 4  | 1001| 0   | 981 | 958 | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   |
| 2             | 4  | 934 | 0   | 1040|1005 | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   |
| 3             | 11 | 0   | 0   | 0   |1020 | 950 | 0   | 0   | 966 | 976 | 945 | 0   | 0   |
| 4             | 5  | 993 | 0   | 0   | 988 | 0   | 0   | 0   | 0   | 0   | 0   | 0   | 0   |
| 5             | 3  | 0   | 0   |1023 | 0   | 0   | 984 |1037 | 0   | 0   | 998 |1029 | 962 |
| 6             | 5  | 0   |1002 | 0   | 0   | 0   | 0   | 0   | 0   | 0   |1025 |1013 |1057 |

La cellule (i, j) du DataFrame résultant contient le nombre d’éléments du DataFrame ayant la catégorie i pour la première colonne (`prod_cat_code`) et la catégorie j pour la deuxième colonne (`prod_subcat_code`).  

Ainsi, il est facile de déterminer, par exemple, que les sous-catégories dominantes de la catégorie 4 sont 1 et 4.  

L’argument `normalize` de `crosstab` permet d’afficher les fréquences sous forme de pourcentages.  
Par exemple, utiliser `normalize=1` normalise le tableau selon **l’axe 1**, c’est-à-dire par colonne :

```python
# Extract the year from the transaction date
column1 = transactions['tran_date'].apply(lambda x: int(x.split('-')[2]))
column2 = transactions['store_type']

pd.crosstab(column1,
            column2,
            normalize=1)
```

Cela produit le DataFrame suivant :

| tran_date | Flagship store | MBR     | TeleShop | e-Shop  |
|-----------|----------------|--------|----------|---------|
| 2011      | 0.291942       | 0.323173 | 0.283699 | 0.306947 |
| 2012      | 0.331792       | 0.322093 | 0.336767 | 0.322886 |
| 2013      | 0.335975       | 0.3115   | 0.332512 | 0.320194 |
| 2014      | 0.0402906      | 0.0432339| 0.0470219| 0.0499731|

Ce DataFrame permet d’affirmer que 33,5975 % des transactions effectuées dans un « magasin phare » ont eu lieu en 2013.  

Inversement, en définissant `normalize=0`, nous normalisons le tableau selon les **lignes** :

| tran_date | Flagship store | MBR     | TeleShop | e-Shop  |
|-----------|----------------|--------|----------|---------|
| 2011      | 0.191121       | 0.21548  | 0.182617 | 0.410781 |
| 2012      | 0.20096        | 0.198693 | 0.20056  | 0.399787 |
| 2013      | 0.205522       | 0.194074 | 0.2      | 0.400404 |
| 2014      | 0.173132       | 0.189215 | 0.198675 | 0.438978 |

La normalisation par ligne permet de déduire que les transactions effectuées dans un « e-Shop » représentent 41,0781 % des transactions en 2011.  

Dans le fichier `covid_tests.csv`, nous disposons d’un jeu de données de 200 tests COVID-19. Les colonnes de ce jeu de données sont :

- `patient_id` : identifiant du patient testé.  
- `test_result` : résultat du test de détection. 1 si le patient est testé positif, 0 sinon.  
- `infected` : 1 si le patient était réellement infecté, 0 sinon.


<center>

### **🔍 Exemple : Analyse des tests COVID-19**

</center>

---

- (a) Charger le jeu de données à partir du fichier `covid_tests.csv`. Le séparateur est `;`.  
- (b) À l’aide de la fonction `pd.crosstab`, déterminer le nombre de **faux négatifs** produits par ce test.  
Un faux négatif se produit lorsque le test indique qu’un patient n’est pas infecté, alors qu’il l’est réellement.  
- (c) Quel est le **taux de faux positifs** du test ?  
Le taux de faux positifs correspond à la proportion de faux positifs parmi tous les individus sains.  
Il faudra normaliser les résultats pour pouvoir le calculer.

In [ ]:
# TODO

---

<center>

## **📖 Conclusion et résumé**

</center>

---

Dans ce notebook, vous avez appris à :

- **Filtrer les lignes d’un DataFrame** avec plusieurs conditions en utilisant les opérateurs binaires `&`, `|` et `~` :

  ```python
  # Année égale à 1979 et surface supérieure à 60
  df[(df['annee'] == 1979) & (df['surface'] > 60)]

  # Année supérieure à 1900 ou quartier égal à 'Père-Lachaise'
  df[(df['année'] > 1900) | (df['quartier'] == 'Père-Lachaise')]
  ```

- **Fusionner des DataFrames** en utilisant la fonction `concat` et la méthode `merge` :

  ```python
  # Concaténation verticale
  pd.concat([df1, df2], axis=0)

  # Concaténation horizontale
  pd.concat([df1, df2], axis=1)

  # Différents types de jointures
  df1.merge(right=df2, on='column', how='inner')
  df1.merge(right=df2, on='column', how='outer')
  df1.merge(right=df2, on='column', how='left')
  df1.merge(right=df2, on='column', how='right')
  ```

- **Trier et ordonner les valeurs d’un DataFrame** en utilisant les méthodes `sort_values` et `sort_index` :

  ```python
  # Trier un DataFrame par la colonne 'column' dans l'ordre croissant
  df.sort_values(by='column', ascending=True)
  ```

- **Effectuer une opération `groupby` complexe** en utilisant des fonctions lambda avec les méthodes `groupby` et `agg` :

  ```python
  functions_to_apply = {
      'column1': ['min', 'max'],
      'column2': [np.mean, np.std],
      'column3': lambda x: x.max() - x.min()
  }

  df.groupby('column_to_group_by').agg(functions_to_apply)
  ```